<a href="https://colab.research.google.com/github/cgangwar11/Coding-Practice/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import spacy
nlp = spacy.load('en')

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pickle
def load_vocab():

  with open('/gdrive/My Drive/stoi.pkl','rb') as f:
    stoi,itos = pickle.load(f)
  return stoi,itos

In [0]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs,embedding_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)



In [0]:
stoi,itos = load_vocab()
INPUT_DIM = len(itos)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5


In [0]:

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [0]:
model.load_state_dict(torch.load('/gdrive/My Drive/basic_9_epochs_87_128_batch.pth'))

In [59]:
model

CNN(
  (embedding): Embedding(25002, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  )
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.5)
)

In [0]:


def predict_sentiment(sentence, min_len=5):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    print(tokenized)
    indexed = []
    for t in tokenized:
      if t in stoi.keys():
        indexed.append(stoi[t])
      else:
        indexed.append(0)
    print(' '.join([itos[i] for i in indexed]))
    tensor = torch.LongTensor(indexed)
    tensor = tensor.unsqueeze(1)
    
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [64]:
model.eval()

CNN(
  (embedding): Embedding(25002, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  )
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.5)
)

In [98]:

torch.onnx.export(trained_model, dummy_input, "output/mnist.onnx")
predict_sentiment("Movie was at all great ")

['Movie', 'was', 'at', 'all', 'great']
Movie was at all great


0.5751968622207642

In [105]:
from torch.autograd import Variable

# Load the trained model from file

dummy_input = Variable(torch.randn(1,6)) 
# Export the trained model to ONNX
 # one black and white 28 x 28 picture will be the input to the model
torch.onnx.export(model,dummy_input,'output.onnx')

RuntimeError: ignored

In [76]:
model.embedding(torch.LongTensor([1,2,3,4,5]).squeeze(1)))

SyntaxError: ignored

In [83]:
model(torch.LongTensor([[1,2,3,4,5] for _ in range(5)]).unsqueeze(1))

RuntimeError: ignored

TypeError: ignored

In [0]:
emb = pickle.load(open('/gdrive/My Drive/embeddings.pkl','rb'))

In [50]:
emb

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.038194, -0.24487 ,  0.72812 , ..., -0.1459  ,  0.8278  ,
         0.27062 ],
       ...,
       [ 0.29367 ,  0.30923 , -0.57261 , ...,  0.31319 ,  1.0634  ,
         0.872   ],
       [ 1.0406  ,  0.21093 , -0.67883 , ...,  0.92809 ,  0.15732 ,
         0.91354 ],
       [ 0.068605, -0.24221 ,  0.27144 , ..., -0.148   , -0.46105 ,
         0.36058 ]], dtype=float32)